
### Created on July, 2023
### Authors: T. A. Biala, Y.O. Afolabi and S. O. Ibrahim

This function implements a rational approximation to the Prabhakar Function (Three Parameter  Mittag Leffler (ML) function) with 1, 2, or 3 parameters. 
The function is named RA which stands for rational approximation. See [1] for a full discussion of the approximation.

###### Inputs
- $z$: the vector (or matrix) whose Prabhakar function we wish to compute
- $m$: the number of terms (to use) of the local series expansion of the Prabhakar function
- $n$: the number of terms of the asymptotic expansion of the Prabhakar function
- $\alpha$: the alpha parameter in the Prabhakar function
- $\beta$: the beta parameter in the Prabhakar function
-  $\gamma$: the gamma parameter in the Prabhakar function. $r=\gamma$ throughout the notebook

###### Outputs:
- Returns a dictionary with 3 items: 
    - parms: the parameters used in the computation of the approximation, see [1].
    - coeffs: the coefficients of the $(\nu, \nu)$-rational approximation, where $\nu = \dfrac{m+n+r-2}{2}$
    - z_RA: the evaluated rational approximation for the elements of $z$


RA($z, m, n, \alpha$) computes the rational approximation with one parameter ($\alpha$) for $z$; $\alpha$ must be real and positive. The one parameter ML function is defined as 
$$ E_{\alpha}(z) = \sum_{k=0}^{\infty} \dfrac{z^k}{\Gamma(\alpha k+1)},$$
where $\Gamma$ is the Euler gamma function.

RA($z, m, n, \alpha, \beta$) computes the rational approximation with two parameters ($\alpha, \beta$) for $z$; $\alpha$ must be real and positive, $\beta$ must be a real scalar. The two parameter ML function is defined as 
$$ E_{\alpha, \beta}(z) = \sum_{k=0}^{\infty} \dfrac{z^k}{\Gamma(\alpha k+\beta)}.$$


RA($z, m, n, \alpha, \beta, \gamma$) computes the rational approximation with three parameters ($\alpha, \beta, \gamma$) for $z$; $\alpha$ must be real and positive, $\beta$ must be a real scalar and $\gamma$ is a positive integer. The three parameter ML function is defined as 
$$ E_{\alpha, \beta}^\gamma(z) = \sum_{k=0}^{\infty} \dfrac{\Gamma(\gamma + k)z^k}{\Gamma(k+1)\Gamma(\alpha k+\beta)}.$$





In [1]:
# Importing Libraries
import numpy as np
import scipy.special as sc
import scipy.linalg as sl
import numpy.linalg as nl
import math
import warnings

###### Helper Functions

- Function **b_coeffs** computes the $\mathbf{b}^r_k$-coefficients, see [1] for details, $$b_{r+j} = \dfrac{(-1)^{j+1}\Gamma(\beta - \alpha r)\Gamma(r+j)}{\Gamma(j+1)\Gamma(r)\Gamma(\alpha j + \beta)}, ~ j=0, 1, 2, \cdots$$
where $b_j = 0, ~j=0, \cdots, r-1$.
- Function **a_coeffs** computes the $\mathbf{a}^r_k$-coefficients, see [1] for details, $$a_{r+j} = \dfrac{(-1)^{j+1}\Gamma(\beta - \alpha r)\Gamma(r+j)}{\Gamma(j+1)\Gamma(r)\Gamma(\beta - \alpha(j+r))}, ~ j=0, 1, 2, \cdots$$
where $a_j = 0, ~j=0, \cdots, r-1$.
- Function **c_coeffs** computes the $\mathbf{c}^r_k$-coefficients, see [1] for details, $$c_{r+j} = \dfrac{(-1)^{j}\Gamma(- \alpha)\Gamma(r+j)}{\Gamma(j+1)\Gamma(r+1)\Gamma(- \alpha(j+r))}, ~ j=0, 1, 2, \cdots$$
where $c_j = 0, ~j=0, \cdots, r-1$.
- Function **w_coeffs** computes the $\mathbf{w}^r_k$-coefficients, see [1] for details, $$w_{r+j} = \dfrac{(-1)^{j+1}\Gamma(- \alpha)\Gamma(r+j+1)}{\Gamma(j+2)\Gamma(r+1)\Gamma(- \alpha(j+1))}, ~ j=0, 1, 2, \cdots$$
where $w_j = 0, ~j=0, \cdots, r-1$.

In [2]:
def b_coeffs(alpha, beta, r, m):
    c_coeff = sc.gamma(beta - alpha * r)/sc.gamma(r)
    j = np.arange(m)
    return  (-1)**(j+1)*sc.gamma(j+r)/(sc.gamma(j+1)*sc.gamma(alpha*j + beta))*c_coeff

def a_coeffs(alpha, beta, r, n):
    c_coeff = sc.gamma(beta - alpha * r)/sc.gamma(r)
    j = np.arange(n)
    return  (-1)**(j+1)*sc.gamma(j+r)/(sc.gamma(j+1)*sc.gamma(beta - alpha*(j+r)))*c_coeff

def c_coeffs(alpha, r, m):
    c_coeff = sc.gamma(-alpha)/sc.gamma(r+1)
    j = np.arange(m)
    return  (-1)**(j)*sc.gamma(j+r)/(sc.gamma(j+1)*sc.gamma(alpha*(j + r)))*c_coeff

def w_coeffs(alpha, r, m):
    c_coeff = sc.gamma(-alpha)/sc.gamma(r+1)
    j = np.arange(m)
    return  (-1)**(j+1)*sc.gamma(j+r+1)/(sc.gamma(j+2)*sc.gamma(-alpha*(j + 1)))*c_coeff

- Function **top_set** uses two other functions: **top_right** and **top_left**
    - **top_left** is an $(m-1, \nu-r)$ (or $(m-2, \nu-r-1)$)-rectangular diagonal matrix with entries $1$ for $\beta \neq \alpha\gamma$  (or $\beta = \alpha\gamma$).
    - **top_right** is a 
        - $(\nu, \nu)$ matrix if $\nu-r-n+1 = 0\implies \nu = m-1$,
        - $(2\nu-r-n+1, \nu)$ matrix if  $\nu-r-n+1 > 0$ for $\beta \neq \alpha\gamma$ or $(2\nu-r-n, \nu)$ matrix if $\nu-r-n> 0$ for $\beta = \alpha\gamma$. 
    - **top_left** and **top_right** are stacked horizontally to obtain **top_set**

In [3]:
def top_left(num, nu, r, equal):
    nur = nu-r if not equal else nu-r-1
    return np.eye(num, nur)

def top_right(b, nu, n, r, equal):
    top =  np.tril(sl.toeplitz(tuple(b[:nu])))
    nunr1 = nu-n-r+1 if not equal else nu-n-r
    if nunr1 > 0:
        ll = [b[i:i+nu][::-1] for i in range(1, nunr1+1)]
        top = np.vstack((top, ll))
    return top      
    
def top_set(num, nu, n, r, b, equal):
    left = top_left(num, nu, r, equal)
    right = top_right(b, nu, n, r, equal)
    return np.hstack((left, right))

- Function **bottom_set** uses two other functions: **bottom_right** and **bottom_left**
    - **bottom_left** is an $(n-1, \nu-r)$ (or $(n-1, \nu-r-1)$)-rectangular antidiagonal matrix with entries $1$ for $\beta \neq \alpha\gamma$  (or $\beta = \alpha\gamma$).
    - **bottom_right** is 
        - $(n-1, n-1)$-lower antitriangular matrix with entries $a_{r+j}, \tiny{j = 0, \cdots, n-2}$ if $\nu-n+1 = 0\implies \nu = n-1$,    
        - $(n-1, \nu)$ which consists of $(n-1, n-1)$-lower antitriangular matrix with entries $a_{r+j}, \tiny{j = 0, \cdots, n-2}$ horizontally stacked with $(n-1, \nu-n+1)$-zero matrix if  $\nu-n+1 > 0$ 
    - **bottom_left** and **bottom_right** are stacked horizontally to obtain **bottom_set**

In [4]:
def bottom_left(num, nu, n, r, equal):
    nur = nu-r if not equal else nu-r-1
    return np.eye(num, nur)[:, ::-1]

def bottom_right(num, nu, n, r, a, equal):
    if equal:
        return np.tril(sl.toeplitz(tuple(a[:nu])))[:num, ::-1] 
    nun1 = nu-n+1
    temp = np.tril(sl.toeplitz(tuple(a[:num])))[:, ::-1]
    if nun1 > 0:
        return np.hstack((np.zeros((num, nun1)), temp))
    return temp
 

def bottom_set(num, nu, n, r, a, equal):
    left = bottom_left(num, nu, n, r, equal)
    right = bottom_right(num, nu, n, r, a, equal)
    return np.hstack((left, right))

- Function **create_matrix** consists of vertically stacked results from **top_set** and **bottom_set**

In [5]:
def create_matrix(nu, m, n, r, a_coeff, b_coeff, equal=False):   
    top_num, bottom_num = m-1 if not equal else m-2, n-1 
    top = top_set(top_num, nu, n, r, b_coeff, equal)
    bottom = bottom_set(bottom_num, nu, n, r, a_coeff, equal)
    return np.vstack((top, bottom))

- Function **create_vector** returns a vector $\in \mathbb{R}^{2\nu-r}$  which consists of 
    - 0-vector $\in \mathbb{R}^{p_1}$, where $p_1=\nu-r$ for $\beta\neq\alpha\gamma$ or $p_1=\nu-r-1$ for $\beta=\alpha\gamma$
    - a vector $\in \mathbb{R}^{1}$ with entry -$1$
    - 0-vector $\in \mathbb{R}^{p_2}$, where $p_2=r-1$ for $\beta\neq\alpha\gamma$ or $p_2=r$ for $\beta=\alpha\gamma$
    - a vector $\in \mathbb{R}^{p_3}$  , where $p_3=\nu-r-n+1$ for $\beta\neq\alpha\gamma$ and $\nu-r-n+1>0$ with coefficients -$b_{r+j}, \tiny{j = 0, \cdots, \nu-r-n}$ or $p_3=\nu-r-n$ for $\beta=\alpha\gamma$ and $\nu-r-n>0$ with coefficients -$c_{r+j}, \tiny{j = 0, \cdots, \nu-r-n}$
    - a vector $\in \mathbb{R}^{n-1}$  with entries -$a_{r+j}, \tiny{j = 1, \cdots, n-1}$ for $\beta\neq\alpha\gamma$ or -$w_{r+j}, \tiny{j = 1, \cdots, n-1}$ for $\beta=\alpha\gamma$

In [6]:
def create_vector(nu, n, r, a, b, equal=False):
    a = list(-a) 
    b = list(-b)
    nunr1 = nu-n-r+1 if not equal else nu-n-r
    nur = nu-r if not equal else nu-r-1
    r1 = r-1 if not equal else r
    temp = [0]*nur + [-1.0] + [0]*r1
    if nunr1 > 0:
        temp += b[:nunr1]
    temp += a[1:n]
    return np.array(temp)

- Function **sol_vars** returns the solution variables as a dictionary with the coefficients $\{p_{_k}, q_{_k}\}$ as keys and their corresponding solution values as values
- Function **RA_approx** returns a vector of  approximations to the elements of $z$ (flattened if matrix)
- Function **RA** returns  a dictionary of a triple consisting of the parameters used in the rational approximation, the coefficients of the rational approximation and a vector or matrix (depending on $z$) of  approximations to the elements of $z$.

In [7]:
def sol_vars(sol, nu, m, n, alpha, beta, r, equal):
    
    coeffs = {'alpha' : alpha,
              'beta' : beta,
              'nu' : nu,
              'm' : m,
              'n' : n,
              'gamma' : r,
              'r' : r}
    sol_dict = {}
    r1 = r if not equal else r+1
    nur = nu-r if not equal else nu-r-1
    for i in range(r1):
        sol_dict['p' + str(i)] = 0.0
    for i in range(nur):
        sol_dict['p' + str(r1+i)] = sol[i]
    for i in range(nu):
        sol_dict['q' + str(i)] = sol[nur+i]
    sol_dict['p' + str(nu)] = 1.0
    sol_dict['q' + str(nu)] = 1.0
    return coeffs, sol_dict

def RA_approx(x, sol_dict, alpha, beta, r, equal):
    nu = int(len(sol_dict)/2)
    num, den = 0, 0
    for i in range(nu-1):
        num += sol_dict['p' + str(i)]*x**i
        den += sol_dict['q' + str(i)]*x**(i)
    num += sol_dict['p' + str(nu-1)]*x**(nu-1)
    den += sol_dict['q' + str(nu-1)]*x**(nu-1)
    den *= (sc.gamma(beta-alpha*r)*x**r if not equal else -sc.gamma(-alpha)/r*x**(1+r))
    return num/den

def RA(z, m, n, alpha, beta=1, gamma=1):
    r = gamma 
    nu =  math.floor((m + n + r - 2)/2)
    assert type(gamma) == int, f'GAMMA = {GAMMA} must be an integer'
    assert (m+n+r)%2 == 0, f'm+n+r is not even, i.e, {m}+{n}+{r} = {m+n+r} is odd'
    if isinstance(z, (list, tuple, np.ndarray)):
        z = np.array(z)
    else:
        raise Exception('z must be of type list, tuple,  or nd.array')
    if (z < 0).any():
        z = np.abs(z)
        warnings.warn('\nInput z contains negative values. The absolute values of z will be used instead.')
    
    # Get the shape of z and flatten for easy computation
    z_shape = z.shape
    z = z.flatten()
    
    # Check if there are zero elements in z and compute this value separately
    zero_indices = np.where(z==0)[0]
    zero_val = 1.0/sc.gamma(beta)
    if zero_indices.shape[0] != 0:
        z = np.delete(z, zero_indices)
        
    if beta != alpha*r:
        assert m > n+r-2, f'n+r-2={n+r-2} must be less than m={m}'
        b_coeff = b_coeffs(alpha, beta, r, m)
        a_coeff = a_coeffs(alpha, beta, r, n)
        equal = False
    else:
        assert m > n+r, f'n+r={n+r} must be less than m={m}'
        assert alpha != 1.0, f'Gamma(-{alpha}) is undefined, so alpha cannot be {alpha} with beta = alpha * r'
        b_coeff = c_coeffs(alpha, r, m)
        a_coeff = w_coeffs(alpha, r, max(nu, n))
        equal = True
        
    A = create_matrix(nu, m, n, r, a_coeff, b_coeff, equal=equal)
    b = create_vector(nu, n, r, a_coeff, b_coeff, equal=equal)
    sol = np.linalg.solve(A, b)
  
    parms, coeffs = sol_vars(sol, nu, m, n, alpha, beta, r, equal)
    sol_vector = RA_approx(z, coeffs, alpha, beta, r, equal)
    
    # insert the ML value of zero elements using the appropriate indices
    for i in zero_indices:
        sol_vector = np.insert(sol_vector, i, zero_val)
        
    # Finally, reshape solution to have same size as input
    z_RA = sol_vector.reshape(z_shape)
    
    return {'parms': parms, 'coeffs': coeffs, 'z_RA': z_RA}


#### References
1. Y. O. Afolabi, T. A. Biala, I. O. Sarumi and B. A. Wade, `Global-Type Rational Approximations of the Three-Parameter Mittag-Leffler Functions Based on Polynomial Bases Functions` Submitted